# **Silver Layer**

In the Silver layer we transform the raw Bronze records into cleaned, daily station-level climate data and structured metadata tables.

**Key steps:**
- Read all Bronze Parquet data and restrict to years `2010–2025`.
- Switch to RDD-based processing and map each record to a simpler structure
- Apply quality filtering:
  - keep only records with empty quality flag (`qflag == ""`);
  - keep only valid/empty ingestion status (`status in ["", "valid"]`);
  - drop missing/sentinel values (`value is not None` and `value != -9999`);
  - keep only the three core elements: **TMAX**, **TMIN**, **PRCP**.
- Deduplicate by `(id, date, element)` and convert raw units to SI:
  - temperatures to °C (`tmax_c`, `tmin_c`, derived `tavg_c`),
  - precipitation to millimetres (`prcp_mm`).
- Aggregate to one row per station–day `(id, date, year)` with:
  - `tmax_c`, `tmin_c`, `tavg_c`, `prcp_mm`.
- Create a Silver DataFrame, parse the date, and add convenience fields:
  - `date` (as `yyyy-MM-dd`), `month`, `day`, `year`.
- Write the Silver fact table as Parquet, partitioned by `year`, to `SILVER_PATH`

**MetaData:**
- Parse `ghcnd-stations.txt` (fixed-width) into a cleaned **stations** table
- Parse `ghcnd-inventory.txt` (fixed-width) into an **inventory** table
- Derive a small **coverage** helper table summarizing the min/max years
  of TMAX/TMIN/PRCP coverage per station.
- Store all metadata as Parquet under `silver_meta`
  (`stations.parquet`, `inventory.parquet`, `coverage.parquet`).

## 01. Spark Session

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("GHCN-Silver-RDD").getOrCreate()
spark.sparkContext.setLogLevel("WARN")

print("Spark version:", spark.version)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/26 19:51:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 3.5.0


## 02. Data Paths and Setup

In [2]:
# Data paths
BRONZE_PATH = "/home/ubuntu/spark-notebooks/project/data/bronze" 
SILVER_PATH = "/home/ubuntu/spark-notebooks/project/data/silver" 

# Year bounds for faster iteration
YEAR_MIN = 2010
YEAR_MAX = 2025

# Elements to keep in first pass
ELEMENTS = {"TMAX", "TMIN", "PRCP"}

VERBOSE_SAMPLES = 5

## 03. Read Bronze and switch to RDD

In [3]:
# Checking bronze data
df_bronze = spark.read.parquet("/home/ubuntu/spark-notebooks/project/data/bronze")
print("Columns:", df_bronze.columns)
df_bronze.select("year").distinct().orderBy("year").show(100)

Columns: ['station', 'date_str', 'element', 'raw_value', 'mflag', 'qflag', 'sflag', 'obstime', '_ingestion_timestamp', '_source', '_status', '_raw_data', 'year']


+----+
|year|
+----+
|2010|
|2011|
|2012|
|2013|
|2014|
|2015|
|2016|
|2017|
|2018|
|2019|
|2020|
|2021|
|2022|
|2023|
|2024|
|2025|
+----+



In [4]:
from pyspark.sql import functions as F

# Read Bronze (partitioned by year). We filter by year on the DF read,
# but we won't use DF operations for cleaning itself.
df_bronze = spark.read.format("parquet").load(BRONZE_PATH)

# Using 'year' column to prune the read for faster iteration.
if "year" in df_bronze.columns:
    df_bronze = df_bronze.where((F.col("year") >= YEAR_MIN) & (F.col("year") <= YEAR_MAX))

print("Bronze columns:", df_bronze.columns)
print("Bronze count:", df_bronze.count())
df_bronze.printSchema()

Bronze columns: ['station', 'date_str', 'element', 'raw_value', 'mflag', 'qflag', 'sflag', 'obstime', '_ingestion_timestamp', '_source', '_status', '_raw_data', 'year']
Bronze count: 587797263
root
 |-- station: string (nullable = true)
 |-- date_str: string (nullable = true)
 |-- element: string (nullable = true)
 |-- raw_value: string (nullable = true)
 |-- mflag: string (nullable = true)
 |-- qflag: string (nullable = true)
 |-- sflag: string (nullable = true)
 |-- obstime: string (nullable = true)
 |-- _ingestion_timestamp: double (nullable = true)
 |-- _source: string (nullable = true)
 |-- _status: string (nullable = true)
 |-- _raw_data: string (nullable = true)
 |-- year: integer (nullable = true)



In [5]:
# Switch to RDD for all cleaning
rdd_raw = df_bronze.rdd
print("First raw record:", rdd_raw.first().asDict())

First raw record: {'station': 'CA001063303', 'date_str': '20180917', 'element': 'TMAX', 'raw_value': '146', 'mflag': None, 'qflag': None, 'sflag': 'C', 'obstime': None, '_ingestion_timestamp': 1764178290.6534057, '_source': 'ghcn_txt', '_status': 'valid', '_raw_data': None, 'year': 2018}


## 04. RDD Cleaning (MapReduce style)

In [6]:
from typing import Dict, Any

def to_int_safe(x):
    try:
        return int(x)
    except:
        return None

def to_rec(row) -> Dict[str, Any]:
    d = row.asDict()
    return {
        "id": d.get("station"),                 
        "date": d.get("date_str"),              
        "element": d.get("element"),
        "value": to_int_safe(d.get("raw_value")), 
        "qflag": (d.get("qflag") or ""),        
        "year": d.get("year"),
        "status": (d.get("_status") or "")      
    }

# Map to simple dicts
rdd1 = rdd_raw.map(to_rec)

# Metrics pre-clean (RDD-only)
total_in = rdd1.count()
elem_counts_pre = (rdd1.map(lambda r: (r["element"], 1)).reduceByKey(lambda a,b: a+b).collect())
print(f"Total rows (bounded): {total_in}")
print("Elements present (pre-clean):", sorted(elem_counts_pre))

# Cleaning
rdd_clean = (
    rdd1
    # keep QA-passed
    .filter(lambda r: r["qflag"] == "")
    # keep valid ingestion status if present
    .filter(lambda r: (r["status"] in ("", "valid")))
    # drop sentinel/missing
    .filter(lambda r: (r["value"] is not None and r["value"] != -9999))
    # keep the three core elements
    .filter(lambda r: r["element"] in ELEMENTS)
)

# Deduplicate (id, date, element)
rdd_unique = (
    rdd_clean
    .map(lambda r: ((r["id"], r["date"], r["element"]), r))
    .reduceByKey(lambda a, b: a)   # keep first
    .map(lambda kv: kv[1])
)

# Unit conversion (SI): /10 for TMAX/TMIN/PRCP
rdd_si = rdd_unique.map(lambda r: {**r, "value_si": r["value"] / 10.0})

kept = rdd_si.count()
print(f"Kept after QA+status+missing+element filters: {kept} ({kept/total_in*100:.1f}%)")

# Show few examples
for rec in rdd_si.take(VERBOSE_SAMPLES):
    print(rec)


Total rows (bounded): 587797263
Elements present (pre-clean): [('ADPT', 2484948), ('ASLP', 2480218), ('ASTP', 2480218), ('AWBT', 2484948), ('AWDR', 141537), ('AWND', 6237038), ('DAEV', 7), ('DAPR', 1578216), ('DASF', 314), ('DATN', 42373), ('DATX', 40999), ('DAWM', 2), ('DWPR', 125145), ('EVAP', 770845), ('FMTM', 333968), ('MDEV', 5), ('MDPR', 1580047), ('MDSF', 1662), ('MDTN', 42372), ('MDTX', 40999), ('MDWM', 2), ('MNPN', 362678), ('MXPN', 365989), ('PGTM', 2014083), ('PRCP', 170899310), ('PSUN', 18956), ('RHAV', 2496957), ('RHMN', 2498781), ('RHMX', 2498781), ('SN02', 3389), ('SN03', 6), ('SN11', 351), ('SN12', 6693), ('SN13', 1087), ('SN14', 396), ('SN21', 30), ('SN22', 30), ('SN23', 30), ('SN31', 104928), ('SN32', 844656), ('SN33', 78544), ('SN34', 1173), ('SN35', 21705), ('SN36', 9729), ('SN51', 32024), ('SN52', 341801), ('SN53', 42189), ('SN54', 982), ('SN55', 16818), ('SN56', 6628), ('SN57', 31), ('SNOW', 73934574), ('SNWD', 49575718), ('SX02', 3330), ('SX03', 6), ('SX11', 351)

Kept after QA+status+missing+element filters: 321942526 (54.8%)


{'id': 'US1Hami1858', 'date': '20181024', 'element': 'PRCP', 'value': 0, 'qflag': '', 'year': 2018, 'status': 'valid', 'value_si': 0.0}
{'id': 'US1INGR0018', 'date': '20181025', 'element': 'PRCP', 'value': 0, 'qflag': '', 'year': 2018, 'status': 'valid', 'value_si': 0.0}
{'id': 'US1WAGH0033', 'date': '20180919', 'element': 'PRCP', 'value': 0, 'qflag': '', 'year': 2018, 'status': 'valid', 'value_si': 0.0}
{'id': 'USR0000OWAG', 'date': '20181020', 'element': 'TMAX', 'value': 161, 'qflag': '', 'year': 2018, 'status': 'valid', 'value_si': 16.1}
{'id': 'JM000078388', 'date': '20181023', 'element': 'TMIN', 'value': 248, 'qflag': '', 'year': 2018, 'status': 'valid', 'value_si': 24.8}


## 05. Combine to daily rows

In [7]:

# Combine to one row per (id, date, year)
rdd_by_day = (
    rdd_si
    .map(lambda r: ((r["id"], r["date"], r["year"]), {r["element"]: r["value_si"]}))
    .reduceByKey(lambda a,b: {**a, **b})
)

def to_daily(kv):
    (id_, date, year) = kv[0]
    bag = kv[1]
    tmax = bag.get("TMAX")
    tmin = bag.get("TMIN")
    prcp = bag.get("PRCP")
    tavg = (tmax + tmin)/2.0 if (tmax is not None and tmin is not None) else None
    return (id_, date, year, tmax, tmin, tavg, prcp)

rdd_daily = rdd_by_day.map(to_daily)

print("Sample daily rows:")
for row in rdd_daily.take(VERBOSE_SAMPLES):
    print(row)
print("Daily row count:", rdd_daily.count())


Sample daily rows:


('USC00227815', '20150416', 2015, 26.1, 16.1, 21.1, 0.5)
('USC00392984', '20210429', 2021, 17.2, 3.3, 10.25, 0.0)
('RSM00031987', '20230328', 2023, 11.4, -4.3, 3.5500000000000003, 0.0)
('USC00408238', '20130704', 2013, 25.6, 18.3, 21.950000000000003, 0.0)
('US1NCML0016', '20230329', 2023, None, None, None, 2.0)


Daily row count: 186418577


## 06. Convert to DataFrame (write only) and Save Silver

In [8]:

from pyspark.sql import types as T

schema = T.StructType([
    T.StructField("id", T.StringType(), False),
    T.StructField("yyyymmdd", T.StringType(), False),
    T.StructField("year", T.IntegerType(), True),
    T.StructField("tmax_c", T.DoubleType(), True),
    T.StructField("tmin_c", T.DoubleType(), True),
    T.StructField("tavg_c", T.DoubleType(), True),
    T.StructField("prcp_mm", T.DoubleType(), True),
])

df_silver = spark.createDataFrame(rdd_daily, schema)

from pyspark.sql import functions as F

# Parse date & add convenience fields
df_silver = (df_silver
    .withColumn("date", F.to_date(F.col("yyyymmdd"), "yyyyMMdd"))
    .withColumn("month", F.month("date"))
    .withColumn("day", F.dayofmonth("date"))
    .drop("yyyymmdd")
)

# Write Silver (Parquet is fine; switch to 'delta' if you prefer)
(df_silver
 .write
 .mode("overwrite")
 .partitionBy("year")
 .format("parquet")
 .save(SILVER_PATH))

print("Silver written to:", SILVER_PATH)


25/11/26 21:25:08 WARN MemoryManager: Total allocation exceeds 95.00% (996,147,188 bytes) of heap memory
Scaling row group sizes to 92.77% for 8 writers
25/11/26 21:25:08 WARN MemoryManager: Total allocation exceeds 95.00% (996,147,188 bytes) of heap memory
Scaling row group sizes to 92.77% for 8 writers
25/11/26 21:25:08 WARN MemoryManager: Total allocation exceeds 95.00% (996,147,188 bytes) of heap memory
Scaling row group sizes to 92.77% for 8 writers
25/11/26 21:25:09 WARN MemoryManager: Total allocation exceeds 95.00% (996,147,188 bytes) of heap memory
Scaling row group sizes to 92.77% for 8 writers
25/11/26 21:25:09 WARN MemoryManager: Total allocation exceeds 95.00% (996,147,188 bytes) of heap memory
Scaling row group sizes to 92.77% for 8 writers
25/11/26 21:25:09 WARN MemoryManager: Total allocation exceeds 95.00% (996,147,188 bytes) of heap memory
Scaling row group sizes to 92.77% for 8 writers
25/11/26 21:25:09 WARN MemoryManager: Total allocation exceeds 95.00% (996,147,188

Silver written to: /home/ubuntu/spark-notebooks/project/data/silver


## 07. Validation & sanity checks

In [9]:

# Read back and verify partitioning & counts
df_check = spark.read.parquet(SILVER_PATH)

print("Silver partitions (years):")
if "year" in df_check.columns:
    df_check.select("year").distinct().orderBy("year").show(40, truncate=False)

print("Silver schema:")
df_check.printSchema()

print("Row count:", df_check.count())

print("Preview:")
df_check.orderBy("date").show(10, truncate=False)


Silver partitions (years):
+----+
|year|
+----+
|2010|
|2011|
|2012|
|2013|
|2014|
|2015|
|2016|
|2017|
|2018|
|2019|
|2020|
|2021|
|2022|
|2023|
|2024|
|2025|
+----+

Silver schema:
root
 |-- id: string (nullable = true)
 |-- tmax_c: double (nullable = true)
 |-- tmin_c: double (nullable = true)
 |-- tavg_c: double (nullable = true)
 |-- prcp_mm: double (nullable = true)
 |-- date: date (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- year: integer (nullable = true)

Row count: 186418577
Preview:


+-----------+------+------+------------------+-------+----------+-----+---+----+
|id         |tmax_c|tmin_c|tavg_c            |prcp_mm|date      |month|day|year|
+-----------+------+------+------------------+-------+----------+-----+---+----+
|USR0000MRED|-2.2  |NULL  |NULL              |NULL   |2010-01-01|1    |1  |2010|
|USC00360475|NULL  |NULL  |NULL              |0.5    |2010-01-01|1    |1  |2010|
|US1NMDA0165|NULL  |NULL  |NULL              |0.0    |2010-01-01|1    |1  |2010|
|MXN00001090|21.7  |5.6   |13.649999999999999|7.4    |2010-01-01|1    |1  |2010|
|USW00093806|NULL  |NULL  |NULL              |1.5    |2010-01-01|1    |1  |2010|
|USC00412772|NULL  |NULL  |NULL              |0.0    |2010-01-01|1    |1  |2010|
|US1TXCLL019|NULL  |NULL  |NULL              |0.0    |2010-01-01|1    |1  |2010|
|US1COJF0125|NULL  |NULL  |NULL              |0.0    |2010-01-01|1    |1  |2010|
|USC00205406|NULL  |NULL  |NULL              |0.5    |2010-01-01|1    |1  |2010|
|US1NYMD0009|NULL  |NULL  |N

## 08. MetaData

In [10]:
# === Silver: parse & persist NOAA metadata (stations + inventory) ===
import os
from pyspark.sql import functions as F, types as T

STATIONS_TXT  = "/home/ubuntu/spark-notebooks/project/data/meta/ghcnd-stations.txt"
INVENTORY_TXT = "/home/ubuntu/spark-notebooks/project/data/meta/ghcnd-inventory.txt"

OUT_DIR = "/home/ubuntu/spark-notebooks/project/data/silver_meta"
STATIONS_PQ  = f"{OUT_DIR}/stations.parquet"
INVENTORY_PQ = f"{OUT_DIR}/inventory.parquet"
COVERAGE_PQ  = f"{OUT_DIR}/coverage.parquet"

os.makedirs(OUT_DIR, exist_ok=True)

# --- Parse ghcnd-stations.txt (fixed width) ---
stn_schema = T.StructType([
    T.StructField("id",    T.StringType(), False),
    T.StructField("lat",   T.DoubleType(), True),
    T.StructField("lon",   T.DoubleType(), True),
    T.StructField("elev",  T.DoubleType(), True),
    T.StructField("state", T.StringType(), True),
    T.StructField("name",  T.StringType(), True),
    T.StructField("gsn",   T.StringType(), True),
    T.StructField("hcn_crn", T.StringType(), True),
    T.StructField("wmo",   T.StringType(), True),
])

stn_rdd = (spark.read.text(STATIONS_TXT).rdd
    .map(lambda r: r[0])
    .filter(lambda s: len(s) >= 85)
    .map(lambda s: (
        s[0:11].strip(),
        float(s[12:20].strip() or "nan"),
        float(s[21:30].strip() or "nan"),
        float(s[31:37].strip() or "nan"),
        s[38:40].strip() or None,
        s[41:71].strip() or None,
        s[72:75].strip() or None,
        s[76:79].strip() or None,
        s[80:85].strip() or None,
    ))
)
stations_df_raw = spark.createDataFrame(stn_rdd, stn_schema).dropDuplicates(["id"])

# Light cleaning: convert NaNs→nulls; clip lat/lon to valid bounds; normalize text
stations_df = (stations_df_raw
    .withColumn("lat",  F.when(F.isnan("lat"),  None).otherwise(F.col("lat")))
    .withColumn("lon",  F.when(F.isnan("lon"),  None).otherwise(F.col("lon")))
    .withColumn("elev", F.when(F.isnan("elev"), None).otherwise(F.col("elev")))
    .filter((F.col("lat").isNull()) | ((F.col("lat") >= -90) & (F.col("lat") <= 90)))
    .filter((F.col("lon").isNull()) | ((F.col("lon") >= -180) & (F.col("lon") <= 180)))
    .withColumn("state", F.upper(F.col("state")))
    .withColumn("name",  F.trim(F.col("name")))
)

(stations_df.write.mode("overwrite").parquet(STATIONS_PQ))

# --- Parse ghcnd-inventory.txt (fixed width) ---
inv_schema = T.StructType([
    T.StructField("id",        T.StringType(), False),
    T.StructField("lat",       T.DoubleType(), True),
    T.StructField("lon",       T.DoubleType(), True),
    T.StructField("element",   T.StringType(), False),
    T.StructField("first_year",T.IntegerType(), True),
    T.StructField("last_year", T.IntegerType(), True),
])

inv_rdd = (spark.read.text(INVENTORY_TXT).rdd
    .map(lambda r: r[0])
    .filter(lambda s: len(s) >= 45)
    .map(lambda s: (
        s[0:11].strip(),
        float(s[12:20].strip() or "nan"),
        float(s[21:30].strip() or "nan"),
        s[31:35].strip(),
        int((s[36:40].strip() or "0")),
        int((s[41:45].strip() or "0")),
    ))
)
inventory_df_raw = spark.createDataFrame(inv_rdd, inv_schema)

inventory_df = (inventory_df_raw
    .withColumn("lat",  F.when(F.isnan("lat"),  None).otherwise("lat"))
    .withColumn("lon",  F.when(F.isnan("lon"),  None).otherwise("lon"))
    .filter(F.col("element").isin("TMAX","TMIN","PRCP","TAVG","SNOW","SNWD"))  # keep common ones (adjust if you like)
)

(inventory_df.write.mode("overwrite").parquet(INVENTORY_PQ))

# --- Tiny helper table used often in Gold: coverage per station for your core elements ---
coverage_df = (inventory_df
    .filter(F.col("element").isin("TMAX","TMIN","PRCP"))
    .groupBy("id")
    .agg(
        F.min("first_year").alias("min_first_year"),
        F.max("last_year").alias("max_last_year")
    )
)

(coverage_df.write.mode("overwrite").parquet(COVERAGE_PQ))

print("Saved:")
print("  stations  ->", STATIONS_PQ)
print("  inventory ->", INVENTORY_PQ)
print("  coverage  ->", COVERAGE_PQ)


25/11/26 21:32:08 WARN MemoryManager: Total allocation exceeds 95.00% (996,147,188 bytes) of heap memory
Scaling row group sizes to 92.77% for 8 writers


Saved:
  stations  -> /home/ubuntu/spark-notebooks/project/data/silver_meta/stations.parquet
  inventory -> /home/ubuntu/spark-notebooks/project/data/silver_meta/inventory.parquet
  coverage  -> /home/ubuntu/spark-notebooks/project/data/silver_meta/coverage.parquet
